In [63]:
 import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (253 kB/s)
Reading package lists... Done


In [64]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [65]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
house_sale_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
house_sale_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [66]:
# 2. Create a temporary view of the DataFrame.
house_sale_df.createOrReplaceTempView('house_sale')


In [67]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("SELECT  ROUND(avg(price),2) AS avg_price, YEAR(date) as year from house_sale WHERE bedrooms = 4 group by year order by year desc").show()

+---------+----+
|avg_price|year|
+---------+----+
|296363.88|2022|
|301819.44|2021|
|298353.78|2020|
| 300263.7|2019|
+---------+----+



In [68]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql('Select ROUND(avg(price),2), date_built AS Year_Built from house_sale WHERE  bedrooms =3 AND bathrooms = 3 AND floors = 2 AND sqft_living >=2000   GROUP BY Year_Built ORDER BY Year_Built desc' ).show()

+--------------------+----------+
|round(avg(price), 2)|Year_Built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



In [69]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("Select ROUND(avg(price),2) AS Avg_price, date_built as Year_Built FROM house_sale WHERE bedrooms =3  AND bathrooms = 3 GROUP BY date_built ORDER BY Year_Built desc " ).show()

+---------+----------+
|Avg_price|Year_Built|
+---------+----------+
|292676.79|      2017|
|290555.07|      2016|
| 288770.3|      2015|
|290852.27|      2014|
|295962.27|      2013|
|293683.19|      2012|
|291117.47|      2011|
|292859.62|      2010|
+---------+----------+



In [70]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql('select ROUND(avg(price),2) AS Avg_price, view from house_sale group by view HAVING AVG(price)>=350000').show()


print("--- %s seconds ---" % (time.time() - start_time)) 

+----------+----+
| Avg_price|view|
+----------+----+
| 788128.21|  51|
| 798684.82|  54|
| 750537.94|  69|
| 1072285.2|  87|
| 752861.18|  73|
| 767036.67|  64|
|  791453.0|  59|
|1056336.74|  85|
| 733780.26|  52|
|  775651.1|  71|
|1053739.33|  98|
|1061201.42|  99|
|1017815.92|  96|
| 1026669.5| 100|
| 695865.58|  70|
| 746877.59|  61|
|1114042.94|  75|
|1080649.37|  78|
|1107839.15|  89|
|1076205.56|  77|
+----------+----+
only showing top 20 rows

--- 0.9791052341461182 seconds ---


In [71]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table house_sale')

DataFrame[]

In [72]:
# 8. Check if the table is cached.
spark.catalog.isCached("house_sale")

True

In [73]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql('select ROUND(avg(price),2) AS Avg_price, view from house_sale group by view HAVING AVG(price)>=350000').show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+----+
| Avg_price|view|
+----------+----+
| 788128.21|  51|
| 798684.82|  54|
| 750537.94|  69|
| 1072285.2|  87|
| 752861.18|  73|
| 767036.67|  64|
|  791453.0|  59|
|1056336.74|  85|
| 733780.26|  52|
|  775651.1|  71|
|1053739.33|  98|
|1061201.42|  99|
|1017815.92|  96|
| 1026669.5| 100|
| 695865.58|  70|
| 746877.59|  61|
|1114042.94|  75|
|1080649.37|  78|
|1107839.15|  89|
|1076205.56|  77|
+----------+----+
only showing top 20 rows

--- 0.49482274055480957 seconds ---


In [74]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
house_sale_df.write.partitionBy("date_built").mode("overwrite").parquet("house_sale")

In [75]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('house_sale')

In [79]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_house_sale_p')

In [82]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql('select ROUND(avg(price),2) AS Avg_price, view from house_sale group by view HAVING AVG(price)>=350000').show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
| Avg_price|view|
+----------+----+
| 788128.21|  51|
| 798684.82|  54|
| 750537.94|  69|
| 1072285.2|  87|
| 752861.18|  73|
| 767036.67|  64|
|  791453.0|  59|
|1056336.74|  85|
| 733780.26|  52|
|  775651.1|  71|
|1053739.33|  98|
|1061201.42|  99|
|1017815.92|  96|
| 1026669.5| 100|
| 695865.58|  70|
| 746877.59|  61|
|1114042.94|  75|
|1080649.37|  78|
|1107839.15|  89|
|1076205.56|  77|
+----------+----+
only showing top 20 rows

--- 0.49187207221984863 seconds ---


In [86]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table house_sale")

DataFrame[]

In [87]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("house_sale")

False